In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import ir_datasets
import random

# Collecting data

In [ ]:
dataset = ir_datasets.load("neuclir/1/multi/trec-2023")

print(dataset.docs_count())
print(dataset.queries_count())
print(dataset.qrels_count())

10038768
76
79934


In [ ]:
qrels_df = pd.DataFrame(dataset.qrels_iter())
qrels_doc_ids = set(qrels_df["doc_id"])
documents_df = pd.DataFrame(dataset.docs_iter())
# Filter documents usine qrel dataset
documents_df = documents_df[documents_df["id"].isin(qrels_doc_ids)]
queries_df = pd.DataFrame(dataset.queries_iter())

In [5]:
documents_df.shape[0]

15338

In [6]:
documents_df.head()

,Unnamed: 0,doc_id,title,text,url,time,cc_file
0,163,14a0a786-c707-487c-af0a-16ef97e1d023,越捷航空連三年獲評為「最佳超低廉航空公司」 - 生活,AirlineRatings頒發越捷航空 2020「最佳超低廉航空公司」殊榮。圖：越捷提供\...,https://www.chinatimes.com/realtimenews/201911...,2019-11-27T15:44:07+08:00,crawl-data/CC-NEWS/2019/11/CC-NEWS-20191127083...
1,232,fc24998f-6ed8-43f5-93aa-f71244cd50af,精品包用酒精消毒小心壞掉！快來學4招正確、實用的「清潔技巧」,疫情期間只要一出門，回到家就要把隨身物品消毒一次，那麼大家都是如何幫心愛的精品包消毒呢？其實...,https://fashion.ettoday.net/news/2024734?from=...,2021-07-08T09:09:00+08:00,crawl-data/CC-NEWS/2021/07/CC-NEWS-20210708003...
2,250,b6c8165f-b19a-4d47-84b9-0c7c7e450cf6,一室枯山水 喝一杯手冲咖啡忘憂 - 20210708 - 副刊,手搖凍啡 入口柔順\n\n一般禪味的café均以茶品為主，不過店家主打咖啡，設有手冲咖啡菜單...,https://news.mingpao.com/pns/%E5%89%AF%E5%88%8...,NaN,crawl-data/CC-NEWS/2021/07/CC-NEWS-20210708003...
3,301,42dfc137-e3b8-4ea5-ba96-6914dc6afc1f,《電子零件》車用、儲能雙引擎 系統電6月營收月增5.97％ - 財經,系統電(5309)因車用及儲能產品需求強勁，6月合併營收為2.13億元，月成長5.97%，系...,https://www.chinatimes.com/realtimenews/202107...,2021-07-08T09:44:29+08:00,crawl-data/CC-NEWS/2021/07/CC-NEWS-20210708022...
4,306,e6887499-98df-4408-a6aa-c002a76cc801,富商遇死劫！小20歲嫩妻崩潰痛哭 遭警識破「全是演戲」,記者張寧倢／編譯\n\n印尼巴布亞省（Papua）於6月28日發生一起劫車殺人的嚴重社會案件...,https://www.ettoday.net/news/20210708/2025480....,2021-07-08T10:40:00+08:00,crawl-data/CC-NEWS/2021/07/CC-NEWS-20210708041...


In [7]:
queries_df.head()

,query_id,title,description,narrative,fa_mt_title,fa_mt_description,fa_mt_narrative,ru_mt_title,ru_mt_description,ru_mt_narrative,zh_mt_title,zh_mt_description,zh_mt_narrative
0,200,Corruption Bribery Sports Federation Olympics,Are there cases of Institutional Corruption an...,We are Looking for articles that contain a cas...,المپیک فدراسیون ورزشی رشوه خواری فساد,آیا مواردی از فساد نهادی و رشوه خواری در ورزش ...,ما به دنبال مقالاتی هستیم که حاوی یک مورد فساد...,Олимпийские игры Федерации спортивных игр корр...,Существуют ли случаи институциональной коррупц...,"Мы ищем статьи, которые содержат случай финанс...",腐败贿赂体育联合会奥运会,体育中是否存在机构腐败和贿赂的案例？,我们正在寻找在机构层面上包含金融腐败或贿赂案件的文章。奥运会和体育联合会中的贿赂案件是相关的...
1,201,China investment in Iran,In what fields or industries China’s investmen...,We need articles that specify at least one fie...,سرمایه گذاری چین در ایران,در کدام زمینه ها یا صنایع سرمایه گذاری چین در ...,ما به مقالاتی نیاز داریم که حداقل یک زمینه سرم...,Китайские инвестиции в Иран,В каких областях или промышленности инвестиции...,"Нам нужны статьи, в которых указано хотя бы од...",中国对伊朗的投资,在哪些领域或行业中，中国对伊朗的投资是？,我们需要至少指定一个投资领域的文章，无论是皮带和道路倡议，石油，水坝等。货币的投资金额不提及...
2,202,"Emerging technologies, precision farming, smar...",What are some of the latest emerging technolog...,Find articles on emerging technologies in prec...,فن آوری های نوظهور ، کشاورزی دقیق ، کشاورزی هو...,برخی از جدیدترین فن آوری های نوظهور که صنعت کش...,مقالاتی در مورد فن آوری های نوظهور در کشاورزی ...,"Новые технологии, точное сельское хозяйство, у...",Каковы некоторые из последних новых технологий...,Найдите статьи о новых технологиях в области т...,新兴技术，精确农业，智能农业，农业,哪些最新的新兴技术正在改变农业行业？,查找有关精确农业和农业中新兴技术的文章，这些文章提供了这些技术如何用于提高农作物产量并提高农...
3,203,Ever Given's Stuck,Find information about the ship Ever Given bei...,Find articles on the event when the container ...,تا به حال گیر کرده است,اطلاعات مربوط به کشتی را که در کانال سوئز گیر ...,مقالاتی را در مورد این رویداد پیدا کنید که کشت...,Когда -либо дал застрял,"Найдите информацию о корабле, когда -либо заст...","Найдите статьи на мероприятии, когда контейнер...",曾经被卡住了,查找有关被困在苏伊士运河的船的信息,在2021年春季遇到的集装箱船被困在苏伊士运河中时，请查找有关事件的事件的文章。包括有关事件...
4,204,Penalty doping sports stop,What strategies are used to reduce/stop doping...,Any mention of the monetary penalties or exclu...,مجازات دوپینگ ورزش توقف,از چه استراتژی هایی برای کاهش/متوقف کردن دوپین...,هرگونه ذکر مجازات های پولی یا محرومیت تیم ها ی...,Спортивная остановка пенальти,Какие стратегии используются для уменьшения/ос...,Любое упоминание о денежных наказаниях или иск...,点球兴奋剂运动停止,哪些策略用于减少运动中的掺杂？,包括任何提及货币罚款或排除在团队中或夺回奖牌的罚款。接受掺杂的测试将不被视为预防措施。


In [ ]:
queries_dt = {}

for lang in ["en"]:
    title_col = f"{lang}_mt_title" if f"{lang}_mt_title" in queries_df.columns else "title"
    desc_col = f"{lang}_mt_description" if f"{lang}_mt_description" in queries_df.columns else "description"
    narrative_col = f"{lang}_mt_narrative" if f"{lang}_mt_narrative" in queries_df.columns else "narrative"

    # Check for missing columns
    queries_df[title_col] = queries_df.get(title_col, "")
    queries_df[desc_col] = queries_df.get(desc_col, "")
    queries_df[narrative_col] = queries_df.get(narrative_col, "")
    # only use title
    queries_dt[lang] = queries_df[["query_id", title_col, desc_col, narrative_col]].assign(
        query_text=queries_df[title_col]
    )[["query_id", "query_text"]]

# check the data
for lang, queries in queries_dt.items():
    print(f"{lang.upper()} Queries:")
    print(queries.head())

EN Queries:
  query_id                                         query_text
0      200      Corruption Bribery Sports Federation Olympics
1      201                           China investment in Iran
2      202  Emerging technologies, precision farming, smar...
3      203                                 Ever Given's Stuck
4      204                         Penalty doping sports stop


# Rank Documents Using mBERT
score the relevance of candidate documents for each query.

In [8]:
# Load mBERT model and tokenizer
model_name = "bert-base-multilingual-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def mbert_rerank_batch(query_text, candidate_docs, model, tokenizer, device, batch_size=16, max_length=512):
    scores = []
    doc_texts = candidate_docs['text'].tolist()
    doc_ids = candidate_docs['doc_id'].tolist()

    for i in range(0, len(doc_texts), batch_size):
        batch_texts = doc_texts[i:i+batch_size]

        queries = [query_text] * len(batch_texts
                                     )
        inputs = tokenizer(
            queries,
            batch_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length
        ).to(device)

        # to save memory skip gradient calculations
        with torch.no_grad():
            outputs = model(**inputs)
            # Extract logits and compute scores
            logits = outputs.logits
            batch_scores = torch.softmax(logits, dim=-1)[:, 1].tolist()
            scores.extend(batch_scores)

    return [{"doc_id": doc_id, "score": score} for doc_id, score in zip(doc_ids, scores)]

In [ ]:
from tqdm import tqdm
from collections import defaultdict

ranked_results = defaultdict(list)
for lang, queries in queries_dt.items():
    print(f"Processing language: {lang}")

    for _, query_row in tqdm(queries.iterrows(), total=len(queries)):
        query_id = query_row["query_id"]
        query_text = query_row["query_text"]
        filtered_docs = documents_df[documents_df['doc_id'].isin(qrels_df[qrels_df['query_id'] == query_id]['doc_id'])]
        if filtered_docs.empty:
            continue
        try:
          ranked_results[(query_id, lang)] = mbert_rerank_batch(query_text, filtered_docs, model, tokenizer, model.device)
        except TypeError as e:
          print(f"Error processing query_id {query_id}")

Processing language: en


 91%|█████████ | 69/76 [31:22<03:57, 33.92s/it]

Error processing query_id 268


100%|██████████| 76/76 [34:49<00:00, 27.49s/it]


In [ ]:
ranked_results

defaultdict(list,
            {('200',
              'en'): [{'doc_id': '5ac60ffe-26b0-4955-901b-9c07247198be',
               'score': 0.4872889518737793}, {'doc_id': 'bdbaa536-6860-41be-81b0-5638bc6e8988',
               'score': 0.5073598623275757}, {'doc_id': '4a7ccfb7-283c-48e8-b6e6-251d3ba110d4',
               'score': 0.48133254051208496}, {'doc_id': '1fefc813-4f11-4adb-a0c1-b14c2d3cb3fa',
               'score': 0.4893096685409546}, {'doc_id': 'ad604d0e-54a3-4ab2-a54c-8f01931af68c',
               'score': 0.4799764156341553}, {'doc_id': 'e186aa5f-9d54-4fa1-be4a-0897c18b5517',
               'score': 0.47846829891204834}, {'doc_id': '2a5ed5e2-060d-4d3a-9eda-b33433441b33',
               'score': 0.4771077036857605}, {'doc_id': 'fc75b771-bd7d-4d1b-acae-6b71855ab0b9',
               'score': 0.4796488881111145}, {'doc_id': 'a208ad3a-91c2-4a7e-bead-e0f5eb4a65a6',
               'score': 0.4859197735786438}, {'doc_id': '4d7d3a52-7955-4d8d-b801-45519615be63',
               'score'

In [ ]:
import json
# Convert tuple keys to string format
ranked_results_dict = {
    f"{query_id}": docs
    for (query_id), docs in ranked_results.items()
}

# Save to JSON file
with open("ranked_results.json", "w") as f:
    json.dump(ranked_results_dict, f, indent=4)

# Evaluation

In [ ]:
import ir_measures
from ir_measures import ScoredDoc, Qrel
import ast

qrels = [
    Qrel(query_id=row['query_id'], doc_id=row['doc_id'], relevance=row['relevance'])
    for _, row in qrels_df.iterrows()
]

results = [
    ScoredDoc(query_id=query_id, doc_id=doc['doc_id'], score=doc['score'])
    for (query_id, _), docs in ranked_results.items()  # Use only query_id from the tuple key
    for doc in docs
]

metrics = [
    ir_measures.nDCG @ 20,
    ir_measures.AP,
    ir_measures.RBP(rel=1),
    ir_measures.R @ 100,
    ir_measures.R @ 1000
]

evaluation = ir_measures.calc_aggregate(metrics, qrels, results)

print("Evaluation Metrics:")
for metric, value in evaluation.items():
    print(f"{metric}: {value:.4f}")


Evaluation Metrics:
nDCG@20: 0.1262
R@1000: 0.9868
AP: 0.1906
RBP(rel=1): 0.1660
R@100: 0.5093
